In [1]:
# allow the notebook to access the parent directory so we can import the other modules
# https://stackoverflow.com/a/35273613
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Data Preparation
-----

### Constants and Folder Paths

In [2]:
import os
dataset_folder_path = os.path.join("..", "files", "dataset")

### Load Data and Split into *Test*, *Train/Valid*

In [3]:
from data.DataSet import DataSet
dataset = DataSet()
dataset.load(dataset_folder_path, test_set_percentage=0, validation_set_percentage=0.3)

In [4]:
print(len(dataset.train_data))
print(len(dataset.test_data))

2500
0


### Data Preprocessing

In [5]:
NUM_SAMPLES = 50
from utils.preprocessing import *
from functools import partial
dataset.apply(apply_mean_centering)
dataset.apply(apply_unit_distance_normalization)
dataset.apply(partial(spline_interpolate_and_resample, num_samples=NUM_SAMPLES))
dataset.expand(reverse_digit_sequence)

In [6]:
print(len(dataset.train_data))
print(len(dataset.test_data))

5000
0


### Extract Data

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train = np.array(dataset.train_data)
X_valid = np.array(dataset.valid_data)
# Convert labels to numpy array and OneHot encode them
encoder, Y_train, Y_valid, _ = dataset.onehot_encode_labels()

----------
# Neural Network Setup and Training
----------

## **Regularized 1024 GRU**

### Parameters

In [8]:
PARAM_NUM_EPOCHS = 30
PARAM_BATCH_SIZE = 300

### Setup Model

In [9]:
from models.regularized_1024_gru import Regularized1024GRU
from utils.evaluation import cross_validate_model

mymodel = Regularized1024GRU(X_train.shape[1:])
mymodel.batch_size = PARAM_BATCH_SIZE
mymodel.num_epochs = PARAM_NUM_EPOCHS

mymodel.initialize()
print(mymodel)

Using TensorFlow backend.




----------

Optimizer: <class 'keras.optimizers.Nadam'>
Batch Size: 300
Number of Epochs: 30

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 50, 1024)          3154944   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 1024)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 1024)              6294528   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
______________________________________________

#### Save Model Summary

In [10]:
mymodel.save_summary(dataset.get_recorded_operations())
mymodel.save_config()

### Train Model

In [11]:
mymodel.train(X_train, Y_train, X_valid, Y_valid)

Train on 5000 samples, validate on 2200 samples
Epoch 1/30
5000/5000 [==============================] - 11s 2ms/step - loss: 2.0358 - categorical_accuracy: 0.2412 - val_loss: 1.8418 - val_categorical_accuracy: 0.3423

Epoch 00001: val_categorical_accuracy improved from -inf to 0.34227, saving model to ../files/checkpoints/1525021298.006101/regularized_1024_gru-01-0.34.hdf5
Epoch 2/30
5000/5000 [==============================] - 10s 2ms/step - loss: 1.5716 - categorical_accuracy: 0.4436 - val_loss: 1.1643 - val_categorical_accuracy: 0.6518

Epoch 00002: val_categorical_accuracy improved from 0.34227 to 0.65182, saving model to ../files/checkpoints/1525021298.006101/regularized_1024_gru-02-0.65.hdf5
Epoch 3/30
5000/5000 [==============================] - 10s 2ms/step - loss: 1.1228 - categorical_accuracy: 0.6272 - val_loss: 1.3080 - val_categorical_accuracy: 0.5391

Epoch 00003: val_categorical_accuracy did not improve
Epoch 4/30
5000/5000 [==============================] - 10s 2ms/step 